<a href="https://colab.research.google.com/github/HoangTranNhatNguyen/Evolutionary-Strategies/blob/main/ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from es import Normal, Expectation, params_to_model, model_to_params, normalize
from utils import test

# Define the evolve method and the network model

In [2]:
def evolve(mu, sigma, npop, data_loader, model, optimizer, criterion, scheduler=None, device='cpu'):
    model.train()
    running_loss = 0
    
    normal = Normal(device)
    expectation = Expectation()
    
    prog_bar = tqdm(data_loader)
    for i, (source, target) in enumerate(prog_bar):
        source = source.to(device)
        target = target.to(device)

        theta, ratio = normal(mu, sigma, npop)
        score = torch.zeros_like(ratio)

        for j in range(npop):
            params_to_model(theta[j], model)
            output = model(source)
            score[j] = criterion(output, target).item()

        loss = expectation(normalize(score), ratio)
        running_loss += score.mean()
        prog_bar.set_description('Loss: {}'.format(running_loss/(i+1)))

        # Back-propagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler != None:
            scheduler.step()

    return running_loss / len(data_loader)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.num_filter1 = 8
        self.num_filter2 = 16
        self.num_padding = 2
        # input is 28x28
        # padding=2 for same padding
        self.conv1 = nn.Conv2d(1, self.num_filter1, 5, padding=self.num_padding)
        # feature map size is 14*14 by pooling
        # padding=2 for same padding
        self.conv2 = nn.Conv2d(self.num_filter1, self.num_filter2, 5, padding=self.num_padding)
        # feature map size is 7*7 by pooling
        self.fc = nn.Linear(self.num_filter2*7*7, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_filter2*7*7)   # reshape Variable
        x = self.fc(x)
        return F.log_softmax(x, dim=-1)

# Load MNIST dataset

In [6]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset1 = datasets.MNIST('.', train=True, download=True, transform = transform)
dataset2 = datasets.MNIST('.', train=False, download=True, transform = transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=128, shuffle=False)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Set up hyper-parameters

In [3]:
model = Net().cuda()
for param in model.parameters():
    param.requires_grad = False

In [4]:
flat_param = torch.cat(model_to_params(model), dim=0)
mu = torch.tensor(flat_param, requires_grad=True, device='cuda')
sigma = torch.full(mu.shape, 0.01, requires_grad=True, device='cuda')
npop = 100
optimizer = torch.optim.AdamW([mu,sigma])
criterion = F.nll_loss

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


# Start the evolutionary process

In [7]:
for epoch in range(10):
    evolve(mu, sigma, npop, train_loader, model, optimizer, criterion, device=mu.device)

In [8]:
params_to_model(mu.detach(), model)
loss, test_outputs, test_targets = test(test_loader, model, device=mu.device)

Loss: 0.0: 100%|██████████| 79/79 [00:01<00:00, 39.61it/s]


In [9]:
import sklearn.metrics as metrics
metrics.accuracy_score(test_targets.cpu(), test_outputs.cpu().argmax(-1))

0.9455